# Reading Data
Now that we have the building blocks, types and functions, in place, we can start working with actual data.  Usually you'd read data from CSV files and from databases.

## Reading from CSV files
The easiest way for you to read data from a CSV file is to use the F# [CSV type provider](https://fsharp.github.io/FSharp.Data/library/CsvProvider.html).  The type provider reads the file and automatically generates types based on the columns in the file, which makes it very easy for you to handle the data.

I have created a [sample file](Policies.csv) looking like this.

```
PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision
1;Pol001;55;R2;m;100;1233,54
1;Pol002;55;R2;m;150;264,23
2;Pol003;32;R3;f;125;5465,32
2;Pol004;32;R3;f;300;475,23
3;Pol005;20;R2;m;400;234
4;Pol006;64;R2;f;167;5343,64
```
As you can see, the file is actually semicolon-separated and the amounts use a comma as decimal-separator (Scandinavian-style).  The CSV type provider lets you configure a lot of things like the separator and the culture (country). You can create the type provider in two ways: Either by using a file as template or by defining the columns manually.  In the first case, the provider will try and guess the type for each column based on the data for that column.  If the provider guesses wrong, you can override the type of each column.

Let us see how that works.

In [2]:
// Start by loading the FSharp.Data package with the CSV type provider.
#r "nuget: FSharp.Data"
open FSharp.Data

[<Literal>]
let csvFilePath = "https://raw.githubusercontent.com/t4rzsan/fsharp-for-actuaries/master/notebooks/Policies.csv"

// The easiest way to configure CsvProvider is to specify the columns as a sample.
type PoliciesProvider = CsvProvider<Sample="PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision", Culture="da-DK", Separators=";">

// Load the Policies.csv file.  This does not have to be the same file as the one used as template.
let policies = PoliciesProvider.Load(csvFilePath).Rows
let firstRow = policies |> Seq.head
display(firstRow.PolicyNumber, firstRow.TechnicalProvision)

Item1,Item2
Pol001,"1233,54"


In lines 10 and 11 you can see how the type provider has automatically created a type representing a row in the CSV file.  The generated type lets you access the values of each row throw the column names.

The question is: Did the type provider make the correct guess for the column types?  Let's have a look at the premium column.

In [3]:
firstRow.Premium.GetType()

System.String

It looks like the type provider thinks the premium is an integer.  It made that guess because currently there are no decimal numbers in the premium column in the file.  By default the provider uses the first 1000 lines to determine the type (you can change that number).  Let us create a new provider where we override that guess, using the `Schema` property.

In [4]:
type PoliciesProvider2 = CsvProvider<Sample="PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision", Culture="da-DK", Separators=";", Schema="Premium=decimal, TechnicalProvision=decimal">
let policies2 = PoliciesProvider2.Load(csvFilePath).Rows
let firstRow2 = policies2 |> Seq.head
firstRow2.Premium.GetType()

System.Decimal

Now the premium is interpreted as a decimal.  Much better!

The `Rows` property is a sequence so you can use all the functions of the `Seq` module to analyse and transform the data, similar to what you have done previously for lists.

## Reading from Microsoft SQL Server
You will often have to read data from a database and Microsoft SQL Server might be one such database server.  If you Google reading from SQL Server using F# you will find a plethora of open source libraries that will help you out.  Below I will show you a plain vanilla example on how to read data using nothing but the built in SQL class from .NET.  We will use [SqlConnection](https://docs.microsoft.com/en-us/dotnet/api/system.data.sqlclient.sqlconnection?view=dotnet-plat-ext-3.1&viewFallbackFrom=netcore-3.0) and its friend [SqlCommand](https://docs.microsoft.com/en-us/dotnet/api/system.data.sqlclient.sqlcommand?view=dotnet-plat-ext-3.1) to open a connection and run a SQL query.  We will use a sequence expression to create a sequence of the rows returned from SQL.

```fsharp
type PolicyNumber = PolicyNumber of string

type Policy =
    {
        PolicyNumber: PolicyNumber;
        Premium: decimal;
    }

let getPolicies () =
    let policies = 
        seq {
            use cn = new SqlConnection(@"Integrated Security=SSPI;Persist Security Info=False;Initial Catalog=YOURDATABASE;Data Source=YOURSQLSERVER")
            use cmd = new SqlCommand("SELECT PolicyNumber, Premium FROM Policy", cn)
            cn.Open()
            use reader = cmd.ExecuteReader()
            while reader.Read() do
                yield {
                    PolicyNumber = (PolicyNumber (string reader.["PolicyNumber"]));
                    Premium = Convert.ToDecimal(reader.["Premium"]);
                }
        }

```

Admittedly, this looks a bit messy and not very functional but it is one way to do it.
***
References:
* CSV Type Provider - https://fsharp.github.io/FSharp.Data/library/CsvProvider.html